# Part One: Senctence Similarity 

To achieve the objective of calculating the similarity between two or more sentences, we need a to calculate a sentence embedding which is a vector of number that represents the sentence meaning.

We can do that by a variety of methods: 
<br>

<h3>First Method:</h3>
Calculating the sum of all word embeddings in each senctence, then normalize the output and calculate the similarity between the two vectors by cosine similarity or covariance or any other similarity or dissimilarity method.

A method like this has the drawback of not considering the context in which a word is used in, so we will use the pretrained model BERT to encode the text after making encoding.

In [35]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install torch
!pip install transformers
!pip install scikit-learn
!pip install scipy
!pip install sentence_transformers
!pip install plotly
!pip install networkx


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
import pandas as pd
from sentence_transformers import SentenceTransformer, util


In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased', output_hidden_states=True)
def get_similarity(text1, text2, model, tokenizer):
    # encode the text
    text1 = text1.lower()
    text2 = text2.lower()
    encoded_input1 = tokenizer(text1, return_tensors='pt')
    encoded_input2 = tokenizer(text2, return_tensors='pt')
    # get the output
    output1 = model(**encoded_input1)
    output2 = model(**encoded_input2)
    # multipy the over the sequence
    out_mul1 = 1
    out_mul2 = 1
    for i, vec in enumerate(output1[0].squeeze()):
        # don't take the first and last token
        if i == 0 or i == len(output1[0].squeeze())-1:
            continue
        out_mul1 = out_mul1+vec
    for i, vec in enumerate(output2[0].squeeze()):
        if i == 0 or i == len(output1[0].squeeze())-1:
            continue
        out_mul2 = out_mul2+vec
    # normalize the output over the mean and std
    out_mul1 = (out_mul1-out_mul1.mean())/out_mul1.std()
    out_mul2 = (out_mul2-out_mul2.mean())/out_mul2.std()
    return(cosine_similarity([out_mul1.detach().numpy()], [out_mul2.detach().numpy()]).item())

get_similarity("apples are my favourite fruit", "you love apples", model, tokenizer)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.7008986473083496

<h3>Second Method</h3>
<p>The second method uses sentence_transformers which is based on the paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks which use a siamese network structure on top of BERT to derive semantically meaningful sentence embeddings.</p>

In [38]:
model_sentence = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
def get_similarity_sentence_transformers(text1, text2, model_sentence):
    query_embedding = model_sentence.encode(text1)
    passage_embedding = model_sentence.encode(text2)
    return(util.dot_score(query_embedding, passage_embedding).item())


In [39]:
get_similarity_sentence_transformers("apples are my favourite fruit", "I love apples",model_sentence)

0.8118329048156738

<h2>Evaluation</h2>

For the evaluation we will use the dataset "SemEval STS Task - multiple years, each covers a bunch of topics that share the same precise similarity metric definition" which can be found here https://github.com/brmson/dataset-sts , we will only use the 2012 version for training.

In [40]:
# read tsv txt file
data = []
open_file = open("MSRpar.txt", "r")
with open_file as f:
    for i, line in enumerate(f):
        data.append([])
        split_line = line.split("\t")
        data[i].append(split_line)
np_data = np.array(data)
np_data = np.squeeze(np_data, axis=1)
df = pd.DataFrame(np_data, columns=["Quality", "Sentence1", "Sentence2"])
# normalize quality between 0 and 1
df["Quality"] = df["Quality"].astype(float)
df["Quality"] = (df["Quality"] - df["Quality"].min()) / (df["Quality"].max() - df["Quality"].min())

In [41]:
# Evaluting first method
first_method = []
for i, row in df.iterrows():
    first_method.append(get_similarity(row["Sentence1"], row["Sentence2"], model, tokenizer))
df["First Method"] = first_method

In [42]:
second_method = []
for i, row in df.iterrows():
    second_method.append(get_similarity_sentence_transformers(row["Sentence1"], row["Sentence2"], model_sentence))
df["Second Method"] = second_method

In [43]:
# normalize the two methods between 0 and 1
df["First Method Norm"] = (df["First Method"] - df["First Method"].min()) / (df["First Method"].max() - df["First Method"].min())
df["Second Method Norm"] = (df["Second Method"] - df["Second Method"].min()) / (df["Second Method"].max() - df["Second Method"].min())

In [44]:
# calculate mean squared error
from sklearn.metrics import mean_squared_error
print("Mean squared error for the first method", mean_squared_error(df["Quality"], df["First Method Norm"]))
print("Mean squared error for the Second method", mean_squared_error(df["Quality"], df["Second Method Norm"]))
# calculate pearson correlation
print("Correlation for the first method", pearsonr(df["Quality"], df["First Method Norm"]))
print("Correlation for the Second method", pearsonr(df["Quality"], df["Second Method Norm"]))

Mean squared error for the first method 0.05139338458375586
Mean squared error for the Second method 0.04185717752027718
Correlation for the first method PearsonRResult(statistic=0.36420126612506554, pvalue=6.107832628014426e-25)
Correlation for the Second method PearsonRResult(statistic=0.5038897340497289, pvalue=1.5341131349433133e-49)


Looks like the second method is the better similarity estimate, now let's go to the second part of the project and make a graph representing the similarity

# Part Two: Graph Construction

In [45]:
import networkx as nx
from matplotlib import pyplot as plt
import plotly.graph_objects as go

In [46]:
# read a txt file and divide it into sentences
path = input("Enter the path of the file: ")
open_file = open(path, "r")
with open_file as f:
    sentences = f.readlines()
    # remove whitespace characters like `\n` at the end of each line
    sentences = [x.strip() for x in sentences]
print(sentences)

['Shakespeare was a famous 17th-century diesel mechanic.', "She learned that water bottles are no longer just to hold liquid, but they're also status symbols.", 'When he encountered maize for the first time, he thought it incredibly corny.', 'Flash photography is best used in full sunlight.', 'Traveling became almost extinct during the pandemic.', 'Abstraction is often one floor above you.', 'They wandered into a strange Tiki bar on the edge of the small beach town.', 'People generally approve of dogs eating cat food but not cats eating dog food.', "He was 100% into fasting with her until he understood that meant he couldn't eat.", 'He went back to the video to see what had been recorded and was shocked at what he saw.', 'The clock within this blog and the clock on my laptop are 1 hour different from each other.', 'She opened up her third bottle of wine of the night.', "I've never seen a more beautiful brandy glass filled with wine.", 'I want more detailed information.', 'The truth is 

In [47]:
# create a similarity matrix
def similarity_matrix_create(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i, sentence1 in enumerate(sentences):
        for j, sentence2 in enumerate(sentences):
            if i == j:
                continue
            else:
                similarity = get_similarity_sentence_transformers(sentence1, sentence2, model_sentence)
                similarity_matrix[i][j] = similarity
    return(similarity_matrix)

This code will construct a a graph with the sentences as nodes and the similarity as edges, you can read the sentences be hovering over the nodes.

In [48]:
# construct a graph from the sentences and calculate the the similarity between the sentences on the edges
def graph_constuction(sentences, threshold):
    graph = nx.Graph()
    graph.add_nodes_from(sentences)
    similarity_matrix = similarity_matrix_create(sentences)
    for row in range(len(similarity_matrix)):
        for col in range(len(similarity_matrix)):
            if similarity_matrix[row][col] > threshold:
                graph.add_edge(sentences[row], sentences[col], weight=similarity_matrix[row][col])
    graph.remove_nodes_from(list(nx.isolates(graph)))
    
    pos = nx.spring_layout(graph)
    labels = {}
    for node in graph.nodes():
        labels[node] = node
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    node_trace = go.Scatter(
        x=[],
        y=[],
        hovertext=[],
        mode='markers',
        hovertemplate='%{hovertext}',
        marker=dict(
            showscale=False,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)
        )
    )
    for node in graph.nodes():
        x, y = pos[node]
        node_trace['x'] += (x,)
        node_trace['y'] += (y,)
        node_trace['hovertext'] += (node,)
    for edge in graph.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += (x0, x1, None)
        edge_trace['y'] += (y0, y1, None)
    edge_labels = nx.get_edge_attributes(graph, 'weight')
    edge_annotations = [
        dict(
            x=(pos[edge[0]][0] + pos[edge[1]][0]) / 2,
            y=(pos[edge[0]][1] + pos[edge[1]][1]) / 2,
            text=f"{edge_labels[edge]:.2f}",
            showarrow=False,
            font=dict(size=10, color='red')
        )
        for edge in graph.edges()
    ]
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Similarity Graph Visualization',
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=edge_annotations,
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                    )
    )
    
    fig.show()
graph_constuction(sentences, 0.3)